# Getting Started

#### Note, this package uses f-strings and is only compatible with python versions 3.6 +

As usual, you will be adding a region in your interface configurations file.

In [ ]:
[app_rest]
username = your_username
password = your_password
endpoint = https://rest.americanphysician.partners
log_location = # currently not part of the config, so leave this one blank for now

Now, you want to install the application

If your configurations are correct, you should be able to initialize a connection and begin querying with no problems. 

In [1]:
import os, sys
sys.path.insert(0, '/home/jp/Projects/apprest_api')
os.environ["INTERFACE_CONF_FILE"] = '/home/jp/interface_secrets.conf' # where to look for your configurations
import apprest_interface as api
rest = api.APPRestConnection()

# Entities

Visit https://rest.americanphysician.partners/ to see what entities are available.

# Methods

### `entity_info` tells you what fields are available on a given entity. 

In [10]:
rest.entity_info('sites')

{'fields': ['id',
  'app_name',
  'hospital_name',
  'paycom_name',
  'billing_name',
  'billing_client_identifier',
  'billing_facility_identifier',
  'region_id',
  'state',
  'region_name',
  'tangier_name',
  'tangier_mid',
  'tangier_scribe',
  'tangier_code',
  'tangier_mid_code',
  'tangier_scribe_code'],
 'related_fields': ['reportedshift']}

### `all` gives a paginated result set of all records for a given entity. 

In [12]:
rest.all('sites')

### `de_paginate` gives a de-paginated  result set of any query.

In [15]:
rest.de_paginate(rest.all('sites'))

### `search` allows you to search on numerous criteria, though the available criteria will not be enumerated here. Just try it out.

In [27]:
rest.search('sites', 'OH')['count']

2

### `query` allows you to query on numerous fields.

In [28]:
len(rest.de_paginate(rest.query('sites', state='OH')))

1

In [33]:
rest.de_paginate(rest.query('sites', state__contains='N'))[0]

{'app_name': 'Maryville TN ED',
 'billing_name': 'Blount Memorial Hospital',
 'hospital_name': 'Blount Memorial Hospital',
 'region_name': 'Eastern Region',
 'state': 'TN',
 'tangier_code': 'BMH-MD',
 'tangier_mid': 'Blount Memorial Hospital-APP',
 'tangier_mid_code': 'BMH-APP',
 'tangier_name': 'Blount Memorial Hospital-MD',
 'tangier_scribe': '',
 'tangier_scribe_code': '',
 'url': 'https://rest.americanphysician.partners/sites/300/'}

In [37]:
rest.de_paginate(rest.query('sites', state__gt='Z')) # no results

[]

In [38]:
rest.query('sites', url='https://rest.americanphysician.partners/sites/300/')

{'app_name': 'Maryville TN ED',
 'billing_name': 'Blount Memorial Hospital',
 'hospital_name': 'Blount Memorial Hospital',
 'region_name': 'Eastern Region',
 'state': 'TN',
 'tangier_code': 'BMH-MD',
 'tangier_mid': 'Blount Memorial Hospital-APP',
 'tangier_mid_code': 'BMH-APP',
 'tangier_name': 'Blount Memorial Hospital-MD',
 'tangier_scribe': '',
 'tangier_scribe_code': '',
 'url': 'https://rest.americanphysician.partners/sites/300/'}

### `add` allows you to add a record.

In [12]:
new_record = rest.add('site', app_name='api test name')
new_record

{'app_name': 'api test name',
 'billing_name': None,
 'hospital_name': None,
 'region_name': None,
 'state': None,
 'tangier_code': None,
 'tangier_mid': None,
 'tangier_mid_code': None,
 'tangier_name': None,
 'tangier_scribe': None,
 'tangier_scribe_code': None,
 'url': 'https://rest.americanphysician.partners/sites/416/'}

In [13]:
import re
pattern = r"""https:\/\/rest\.americanphysician\.partners\/[a-zA-Z]*\/(?P<entity_id>\w*)\/"""
regex = re.compile(pattern)
new_url = new_record['url']
print(new_url)
new_entity_id = regex.search(new_url)['entity_id']
print(new_entity_id) # using regex to capture the id

https://rest.americanphysician.partners/sites/416/
416


### `edit` allows you to edit a record.

In [14]:
rest.edit('site', entity_id=new_entity_id, billing_name='api test billing name')

{'app_name': 'api test name',
 'billing_name': 'api test billing name',
 'hospital_name': None,
 'region_name': None,
 'state': None,
 'tangier_code': None,
 'tangier_mid': None,
 'tangier_mid_code': None,
 'tangier_name': None,
 'tangier_scribe': None,
 'tangier_scribe_code': None,
 'url': 'https://rest.americanphysician.partners/sites/416/'}

In [15]:
rest.edit('site', url=new_url, state='TN')

{'app_name': 'api test name',
 'billing_name': 'api test billing name',
 'hospital_name': None,
 'region_name': None,
 'state': 'TN',
 'tangier_code': None,
 'tangier_mid': None,
 'tangier_mid_code': None,
 'tangier_name': None,
 'tangier_scribe': None,
 'tangier_scribe_code': None,
 'url': 'https://rest.americanphysician.partners/sites/416/'}

### `delete` allows you to delete a record.

In [6]:
to_delete = rest.de_paginate(rest.query('site', app_name='api test name'))
for site in to_delete:
    print(rest.delete('site', url=site['url']))  # also works via entity_id

{'Success 204': 'Record deleted successfully.'}


### `get_or_error` is a helper method that wraps `query` to raise an error when any number of results other than 1 is returned.

In [17]:
test_url = rest.de_paginate(rest.all('sites'))[-1]['url']
rest.get_or_error('site', url=test_url)

{'app_name': 'Lancaster SC HM',
 'billing_name': '',
 'hospital_name': 'Springs Memorial Hospital',
 'region_name': '',
 'state': '',
 'tangier_code': '',
 'tangier_mid': '',
 'tangier_mid_code': '',
 'tangier_name': '',
 'tangier_scribe': '',
 'tangier_scribe_code': '',
 'url': 'https://rest.americanphysician.partners/sites/384/'}

This potentially works with anything that works with query.

In [18]:
rest.get_or_error('site', state='TN') # intentionally fail to return because there are too many matchinf records

APICallError: 4 records match (site={"state": "TN"}). Exactly 1 required for get request.

### `get_or_create` is a helper method that wraps `query` which either creates a new entry or gets an existing one  with the exact specified kwargs. (so be careful)

In [5]:
record, created = rest.get_or_create('site', app_name='api test name')
if created:
    print('created', record)
else:
    print('already existed', record)
record, created = rest.get_or_create('site', app_name='api test name')
if created:
    print('created', record)
else:
    print('already existed', record)   

created {'url': 'https://rest.americanphysician.partners/sites/418/', 'app_name': 'api test name', 'hospital_name': None, 'billing_name': None, 'region_name': None, 'state': None, 'tangier_name': None, 'tangier_mid': None, 'tangier_scribe': None, 'tangier_code': None, 'tangier_mid_code': None, 'tangier_scribe_code': None}
already existed {'url': 'https://rest.americanphysician.partners/sites/418/', 'app_name': 'api test name', 'hospital_name': None, 'billing_name': None, 'region_name': None, 'state': None, 'tangier_name': None, 'tangier_mid': None, 'tangier_scribe': None, 'tangier_code': None, 'tangier_mid_code': None, 'tangier_scribe_code': None}


### `add_file` allows you to upload a file.

In [3]:
rest.add_file(file='/home/jp/Projects/apprest_interface/Documentation.ipynb', name="documentation notebook")

{'created': '2017-12-15T14:17:18',
 'file': 'https://zappa-app-rest.s3.amazonaws.com/content/uploaded_files/Documentation.ipynb',
 'name': 'documentation notebook',
 'updated': '2017-12-15T14:17:18',
 'url': 'https://rest.americanphysician.partners/files/1/'}